# Install Packages

In [2]:
pip install ipython-sql==0.5.0 prettytable==3.9.0 pandas numpy



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Import data

In [ ]:
# 1. Import dependencies
import pandas as pd
from sqlalchemy import create_engine

# 2. Load CSV into a DataFrame
df = pd.read_csv('synthetic_meta_ads_eco_supplements.csv')

# 3. Create (or connect) SQLite database
engine = create_engine('sqlite:///marketing.db')

# 4. Write the DataFrame to SQL, replacing the table if it exists
df.to_sql('meta_ads_data', con=engine, if_exists='replace', index=False)

# 5. Load the SQL magic extension (to run SQL directly in Jupyter)
%load_ext sql

# 6.Connect to the SQLite database
%sql sqlite:///marketing.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [22]:
%sql SELECT * FROM meta_ads_data LIMIT 5;

 * sqlite:///marketing.db
Done.


Date,Campaign,Ad_Set,Ad,Country,Age_Group,Gender,Impressions,Clicks,CTR,CPC,Spend,Conversions,Cost_per_Conversion,Revenue
2025-01-01,New Year Boost,New Year Boost - Ad Set 1,New Year Boost - Ad Set 1 - Ad 1,Netherlands,18-24,Male,4952,611,0.1233844911147011,1.19,761.6947468905493,15,76.17,510
2025-01-01,New Year Boost,New Year Boost - Ad Set 1,New Year Boost - Ad Set 1 - Ad 1,Netherlands,18-24,Female,5125,570,0.1112195121951219,1.34,968.1383136722654,42,34.58,1512
2025-01-01,New Year Boost,New Year Boost - Ad Set 1,New Year Boost - Ad Set 1 - Ad 1,Netherlands,25-34,Male,5022,365,0.0726802070888092,1.09,399.2448996548187,46,12.88,992
2025-01-01,New Year Boost,New Year Boost - Ad Set 1,New Year Boost - Ad Set 1 - Ad 1,Netherlands,25-34,Female,5023,400,0.0796336850487756,1.12,496.98518467628145,30,24.85,1000
2025-01-01,New Year Boost,New Year Boost - Ad Set 1,New Year Boost - Ad Set 1 - Ad 1,Netherlands,35-44,Male,5055,558,0.1103857566765578,1.11,650.8920360999524,49,19.72,1881


In [23]:
%sql SELECT COUNT(*) AS row_count, MIN(Date) AS start_date, MAX(Date) AS end_date, COUNT(DISTINCT Date) AS num_days FROM meta_ads_data;

 * sqlite:///marketing.db
Done.


row_count,start_date,end_date,num_days
208512,2025-01-01,2025-06-30,181


Exploratory data analysis --> ETL --> DASHBOARD --> AI REPORT


### 1. Table Overview

In [24]:
%sql SELECT COUNT(*) AS row_count, MIN(Date) AS start_date, MAX(Date) AS end_date, COUNT(DISTINCT Date) AS num_days, COUNT(DISTINCT Campaign) AS num_campaigns, COUNT(DISTINCT Ad_Set) AS num_ad_sets, COUNT(DISTINCT Ad) AS num_ads, COUNT(DISTINCT Country) AS num_countries, COUNT(DISTINCT Age_Group) AS num_age_groups, COUNT(DISTINCT Gender) AS num_genders FROM meta_ads_data;


 * sqlite:///marketing.db
Done.


row_count,start_date,end_date,num_days,num_campaigns,num_ad_sets,num_ads,num_countries,num_age_groups,num_genders
208512,2025-01-01,2025-06-30,181,4,12,36,4,4,2


### 2. Null / Completeness Checks

In [25]:
%sql SELECT COUNT(*) AS total_rows, COUNT(Campaign) AS campaign_not_null, COUNT(Ad_Set) AS ad_set_not_null, COUNT(Ad) AS ad_not_null, COUNT(Impressions) AS impressions_not_null, COUNT(Clicks) AS clicks_not_null, COUNT(Spend) AS spend_not_null, COUNT(Conversions) AS conversions_not_null, COUNT(Revenue) AS revenue_not_null FROM meta_ads_data;


 * sqlite:///marketing.db
Done.


total_rows,campaign_not_null,ad_set_not_null,ad_not_null,impressions_not_null,clicks_not_null,spend_not_null,conversions_not_null,revenue_not_null
208512,208512,208512,208512,208512,208512,208512,208512,208512


### 3. Categorical Consistency

In [26]:
%sql SELECT Gender, COUNT(*) AS count FROM meta_ads_data GROUP BY Gender ORDER BY Gender;


 * sqlite:///marketing.db
Done.


Gender,count
Female,104256
Male,104256


In [28]:
%sql SELECT Age_Group, COUNT(*) AS count FROM meta_ads_data GROUP BY Age_Group ORDER BY Age_Group;


 * sqlite:///marketing.db
Done.


Age_Group,count
18-24,52128
25-34,52128
35-44,52128
45-54,52128


In [27]:
%sql SELECT Country, COUNT(*) AS count FROM meta_ads_data GROUP BY Country ORDER BY Country;


 * sqlite:///marketing.db
Done.


Country,count
Belgium,52128
Denmark,52128
Germany,52128
Netherlands,52128


### 4. Metric Integrity Checks

In [29]:
%sql SELECT COUNT(*) AS bad_ctr_rows FROM meta_ads_data WHERE Impressions > 0 AND ABS((Clicks * 1.0 / Impressions) - CTR) > 0.01;


 * sqlite:///marketing.db
Done.


bad_ctr_rows
6273


In [30]:
%sql SELECT COUNT(*) AS bad_cpc_rows FROM meta_ads_data WHERE Clicks > 0 AND ABS((Spend / Clicks) - CPC) > 0.01;


 * sqlite:///marketing.db
Done.


bad_cpc_rows
174404


### 5. Range / Outlier Checks

In [31]:
%sql SELECT MIN(Impressions) AS min_impressions, MAX(Impressions) AS max_impressions, MIN(Clicks) AS min_clicks, MAX(Clicks) AS max_clicks, MIN(Spend) AS min_spend, MAX(Spend) AS max_spend, MIN(Conversions) AS min_conversions, MAX(Conversions) AS max_conversions, MIN(Revenue) AS min_revenue, MAX(Revenue) AS max_revenue FROM meta_ads_data;


 * sqlite:///marketing.db
Done.


min_impressions,max_impressions,min_clicks,max_clicks,min_spend,max_spend,min_conversions,max_conversions,min_revenue,max_revenue
869,5323,0,1458,0.0,3194.77753091004,0,596,0,23760


In [32]:
%sql SELECT * FROM meta_ads_data WHERE CTR < 0 OR CTR > 1 OR CPC < 0 OR Spend < 0 LIMIT 20;


 * sqlite:///marketing.db
Done.


Date,Campaign,Ad_Set,Ad,Country,Age_Group,Gender,Impressions,Clicks,CTR,CPC,Spend,Conversions,Cost_per_Conversion,Revenue


### 6. Daily Volume Distribution

In [33]:
%sql SELECT Date, COUNT(*) AS rows_per_day FROM meta_ads_data GROUP BY Date ORDER BY Date;


 * sqlite:///marketing.db
Done.


Date,rows_per_day
2025-01-01,1152
2025-01-02,1152
2025-01-03,1152
2025-01-04,1152
2025-01-05,1152
2025-01-06,1152
2025-01-07,1152
2025-01-08,1152
2025-01-09,1152
2025-01-10,1152


### 7. Hierarchy Integrity

In [34]:
%sql SELECT Campaign, COUNT(DISTINCT Ad_Set) AS ad_sets, COUNT(DISTINCT Ad) AS ads FROM meta_ads_data GROUP BY Campaign;


 * sqlite:///marketing.db
Done.


Campaign,ad_sets,ads
Eco Supplement Awareness,3,9
New Year Boost,3,9
Retargeting Push,3,9
Spring Wellness Promo,3,9


### 8. Revenue vs Conversions Check

In [35]:
%sql SELECT COUNT(*) AS zero_revenue_with_conversions FROM meta_ads_data WHERE Conversions > 0 AND Revenue = 0;


 * sqlite:///marketing.db
Done.


zero_revenue_with_conversions
0


### 9. Aggregation / Summation Sanity

In [37]:
%sql SELECT Date, Campaign, Ad_Set, Ad, Country, Age_Group, Gender, SUM(Impressions) AS total_impressions, SUM(Clicks) AS total_clicks, SUM(Spend) AS total_spend, SUM(Conversions) AS total_conversions, SUM(Revenue) AS total_revenue FROM meta_ads_data GROUP BY Date, Campaign, Ad_Set, Ad, Country, Age_Group, Gender ORDER BY Date, Campaign, Ad_Set, Ad LIMIT 20;


 * sqlite:///marketing.db
Done.


Date,Campaign,Ad_Set,Ad,Country,Age_Group,Gender,total_impressions,total_clicks,total_spend,total_conversions,total_revenue
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Belgium,18-24,Female,2501,115,234.1182836294413,9,297
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Belgium,18-24,Male,2470,91,164.78890355659547,1,48
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Belgium,25-34,Female,2517,444,796.2012365808105,7,329
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Belgium,25-34,Male,2551,256,355.3694232679556,18,936
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Belgium,35-44,Female,2539,0,0.0,0,0
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Belgium,35-44,Male,2524,242,230.03301061637256,8,384
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Belgium,45-54,Female,2453,102,87.26008147722797,21,1071
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Belgium,45-54,Male,2459,0,0.0,0,0
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Denmark,18-24,Female,1000,73,61.55272765954744,8,384
2025-01-01,Eco Supplement Awareness,Eco Supplement Awareness - Ad Set 1,Eco Supplement Awareness - Ad Set 1 - Ad 1,Denmark,18-24,Male,1050,61,144.11339849090842,0,0


# ETL

## Extract (done at the top of this page)

In [13]:
print(df.head())

         Date        Campaign                     Ad_Set  \
0  2025-01-01  New Year Boost  New Year Boost - Ad Set 1   
1  2025-01-01  New Year Boost  New Year Boost - Ad Set 1   
2  2025-01-01  New Year Boost  New Year Boost - Ad Set 1   
3  2025-01-01  New Year Boost  New Year Boost - Ad Set 1   
4  2025-01-01  New Year Boost  New Year Boost - Ad Set 1   

                                 Ad      Country Age_Group  Gender  \
0  New Year Boost - Ad Set 1 - Ad 1  Netherlands     18-24    Male   
1  New Year Boost - Ad Set 1 - Ad 1  Netherlands     18-24  Female   
2  New Year Boost - Ad Set 1 - Ad 1  Netherlands     25-34    Male   
3  New Year Boost - Ad Set 1 - Ad 1  Netherlands     25-34  Female   
4  New Year Boost - Ad Set 1 - Ad 1  Netherlands     35-44    Male   

   Impressions  Clicks       CTR   CPC       Spend  Conversions  \
0         4952     611  0.123384  1.19  761.694747           15   
1         5125     570  0.111220  1.34  968.138314           42   
2         5022   

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208512 entries, 0 to 208511
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Date                 208512 non-null  object 
 1   Campaign             208512 non-null  object 
 2   Ad_Set               208512 non-null  object 
 3   Ad                   208512 non-null  object 
 4   Country              208512 non-null  object 
 5   Age_Group            208512 non-null  object 
 6   Gender               208512 non-null  object 
 7   Impressions          208512 non-null  int64  
 8   Clicks               208512 non-null  int64  
 9   CTR                  208512 non-null  float64
 10  CPC                  208512 non-null  float64
 11  Spend                208512 non-null  float64
 12  Conversions          208512 non-null  int64  
 13  Cost_per_Conversion  147089 non-null  float64
 14  Revenue              208512 non-null  int64  
dtypes: float64(4), in

## Transform (Clean & Add Dimensions)

Clean

In [14]:
import numpy as np

# Change data types
df['Date'] = pd.to_datetime(df['Date'])
cat_cols = ['Campaign', 'Ad_Set', 'Ad', 'Country', 'Age_Group', 'Gender']
df[cat_cols] = df[cat_cols].astype('category')

# Add key metrics
df['Profit'] = df['Revenue'] - df['Spend']
df['Conversion_Rate'] = df['Conversions'] / df['Clicks'].replace(0, np.nan)
df['ROAS'] = df['Revenue'] / df['Spend'].replace(0, np.nan)
df['ROI'] = (df['Revenue'] - df['Spend']) / df['Spend'].replace(0, np.nan)

# Extract time features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month_Name'] = df['Date'].dt.month_name()
df['Week'] = df['Date'].dt.isocalendar().week
df['Day_of_Week'] = df['Date'].dt.day_name()



In [15]:
print(df.head())

        Date        Campaign                     Ad_Set  \
0 2025-01-01  New Year Boost  New Year Boost - Ad Set 1   
1 2025-01-01  New Year Boost  New Year Boost - Ad Set 1   
2 2025-01-01  New Year Boost  New Year Boost - Ad Set 1   
3 2025-01-01  New Year Boost  New Year Boost - Ad Set 1   
4 2025-01-01  New Year Boost  New Year Boost - Ad Set 1   

                                 Ad      Country Age_Group  Gender  \
0  New Year Boost - Ad Set 1 - Ad 1  Netherlands     18-24    Male   
1  New Year Boost - Ad Set 1 - Ad 1  Netherlands     18-24  Female   
2  New Year Boost - Ad Set 1 - Ad 1  Netherlands     25-34    Male   
3  New Year Boost - Ad Set 1 - Ad 1  Netherlands     25-34  Female   
4  New Year Boost - Ad Set 1 - Ad 1  Netherlands     35-44    Male   

   Impressions  Clicks       CTR  ...  Revenue       Profit  Conversion_Rate  \
0         4952     611  0.123384  ...      510  -251.694747         0.024550   
1         5125     570  0.111220  ...     1512   543.861686     

Dimension tables

In [16]:
# Create Dimension Tables
# Dim_Date
dim_date = df[['Date', 'Year', 'Month', 'Month_Name', 'Week', 'Day_of_Week']].drop_duplicates().reset_index(drop=True)
dim_date['Date_ID'] = range(1, len(dim_date)+1)

# Dim_Campaign
dim_campaign = df[['Campaign']].drop_duplicates().reset_index(drop=True)
dim_campaign['Campaign_ID'] = range(1, len(dim_campaign)+1)

# Dim_Ad_Set
dim_ad_set = df[['Ad_Set', 'Campaign']].drop_duplicates().reset_index(drop=True)
dim_ad_set = dim_ad_set.merge(dim_campaign, on='Campaign', how='left')
dim_ad_set['Ad_Set_ID'] = range(1, len(dim_ad_set)+1)

# Dim_Ad
dim_ad = df[['Ad', 'Ad_Set']].drop_duplicates().reset_index(drop=True)
dim_ad = dim_ad.merge(dim_ad_set[['Ad_Set_ID','Ad_Set']], on='Ad_Set', how='left')
dim_ad['Ad_ID'] = range(1, len(dim_ad)+1)

# Dim_Country
dim_country = df[['Country']].drop_duplicates().reset_index(drop=True)
dim_country['Country_ID'] = range(1, len(dim_country)+1)

# Dim_Age_Group
dim_age = df[['Age_Group']].drop_duplicates().reset_index(drop=True)
dim_age['Age_Group_ID'] = range(1, len(dim_age)+1)

# Dim_Gender
dim_gender = df[['Gender']].drop_duplicates().reset_index(drop=True)
dim_gender['Gender_ID'] = range(1, len(dim_gender)+1)


In [23]:
print(dim_gender.head())

   Gender  Gender_ID
0    Male          1
1  Female          2


In [24]:
# Create Fact Table

fact = df.merge(dim_date[['Date_ID','Date']], on='Date', how='left') \
         .merge(dim_campaign[['Campaign_ID','Campaign']], on='Campaign', how='left') \
         .merge(dim_ad_set[['Ad_Set_ID','Ad_Set']], on='Ad_Set', how='left') \
         .merge(dim_ad[['Ad_ID','Ad']], on='Ad', how='left') \
         .merge(dim_country[['Country_ID','Country']], on='Country', how='left') \
         .merge(dim_age[['Age_Group_ID','Age_Group']], on='Age_Group', how='left') \
         .merge(dim_gender[['Gender_ID','Gender']], on='Gender', how='left')

fact = fact[['Date_ID','Campaign_ID','Ad_Set_ID','Ad_ID','Country_ID','Age_Group_ID','Gender_ID',
             'Impressions','Clicks','CTR','CPC','Spend','Conversions','Cost_per_Conversion',
             'Revenue','Profit','ROI','Conversion_Rate','ROAS']]


In [25]:
print(fact.head())

   Date_ID  Campaign_ID  Ad_Set_ID  Ad_ID  Country_ID  Age_Group_ID  \
0        1            1          1      1           1             1   
1        1            1          1      1           1             1   
2        1            1          1      1           1             2   
3        1            1          1      1           1             2   
4        1            1          1      1           1             3   

   Gender_ID  Impressions  Clicks       CTR   CPC       Spend  Conversions  \
0          1         4952     611  0.123384  1.19  761.694747           15   
1          2         5125     570  0.111220  1.34  968.138314           42   
2          1         5022     365  0.072680  1.09  399.244900           46   
3          2         5023     400  0.079634  1.12  496.985185           30   
4          1         5055     558  0.110386  1.11  650.892036           49   

   Cost_per_Conversion  Revenue       Profit       ROI  Conversion_Rate  \
0                76.17      5

## Load (Get files to PBI)

In [26]:
import os

# Create folder if it doesn't exist
folder = 'meta_ads_data'
os.makedirs(folder, exist_ok=True)

# Export CSVs into that folder
fact.to_csv(f'{folder}/fact_meta_ads.csv', index=False)
dim_date.to_csv(f'{folder}/dim_date.csv', index=False)
dim_campaign.to_csv(f'{folder}/dim_campaign.csv', index=False)
dim_ad_set.to_csv(f'{folder}/dim_ad_set.csv', index=False)
dim_ad.to_csv(f'{folder}/dim_ad.csv', index=False)
dim_country.to_csv(f'{folder}/dim_country.csv', index=False)
dim_age.to_csv(f'{folder}/dim_age.csv', index=False)
dim_gender.to_csv(f'{folder}/dim_gender.csv', index=False)

print(f"Fact and dimension tables exported into folder '{folder}'!")


Fact and dimension tables exported into folder 'meta_ads_data'!
